In [10]:
# Import Libraries
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

In [11]:
# Code starts here
# Read the file
df = pd.read_csv(r"C:\Users\tdhoble\Downloads\cash_deposit_prediction.csv")

# First 5 rows
df.head()

# Clean the column names
df.columns = df.columns.str.lower().str.replace(' ','_')

# Replace 'NaN'
df.replace('NaN',np.nan,inplace=True)

print(df.isnull().sum())
# Code ends here

serial_number          0
main_office            0
branch_number          0
established_date       0
acquired_date       1493
city                   0
county                 0
state                  0
2010_deposits        740
2011_deposits        578
2012_deposits        329
2013_deposits        175
2014_deposits         56
2015_deposits         19
2016_deposits          0
dtype: int64


In [12]:

df.set_index(keys='serial_number',inplace=True,drop=True)

# Code starts here

df[['established_date','acquired_date']] = df[['established_date','acquired_date']].apply(pd.to_datetime)

y = df['2016_deposits']
X = df.drop('2016_deposits', axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 3) 
# Code ends here

In [13]:
# time_col = X_train.select_dtypes(exclude=[np.number,'O']).columns
time_col = ['established_date', 'acquired_date']

# Code starts here
for dataframe in [X_train, X_val]:
    for col_name in time_col:
        new_col_name = "since_"+col_name
        dataframe[new_col_name] = pd.datetime.now() - dataframe[col_name]
        dataframe[new_col_name] = dataframe[new_col_name].apply(lambda x: float(x.days)/365)
        dataframe.drop(col_name, axis=1, inplace=True)
# Code ends here

In [14]:
cat = X_train.select_dtypes(include='O').columns.tolist()

# Code starts here
X_train.fillna(0, inplace=True)
X_val.fillna(0, inplace=True)

le = LabelEncoder()
for df in [X_train, X_val]:
    for col in cat:
        df[col] = le.fit_transform(df[col])

X_train_temp = pd.get_dummies(data = X_train, columns = cat)
X_val_temp = pd.get_dummies(data = X_val, columns = cat)

print(X_train_temp.head())
# Code ends here

               main_office  branch_number  2010_deposits  2011_deposits  \
serial_number                                                             
2606                     0           6509        19374.0        23293.0   
2001                     0           5415        33042.0        34507.0   
2768                     0           6774         8855.0        18624.0   
1724                     0           4969       109197.0       132822.0   
1120                     0           4317        14274.0        15475.0   

               2012_deposits  2013_deposits  2014_deposits  2015_deposits  \
serial_number                                                               
2606                 26564.0        28966.0        33219.0        38892.0   
2001                 39912.0        65500.0        74012.0        76850.0   
2768                 24435.0        54422.0        63150.0        79338.0   
1724                127834.0       132220.0       146477.0       159482.0   
1120        

In [15]:
# Code starts here
dt = DecisionTreeRegressor(random_state = 5)
dt.fit(X_train, y_train)

accuracy = dt.score(X_val, y_val)

y_pred = dt.predict(X_val)
rmse = np.sqrt(mean_squared_error(y_pred, y_val))

print("rmse for DecisionTreeRegressor: ",rmse)

rmse for DecisionTreeRegressor:  68372.52930639003


In [16]:
from xgboost import XGBRegressor


# Code starts here
xgb = XGBRegressor(max_depth=50, learning_rate=0.83, n_estimators=100)

xgb.fit(X_train, y_train)
accuracy = xgb.score(X_val, y_val)

y_pred = xgb.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_pred, y_val))

print("rmse for XGBRegressor: ",rmse)
# Code ends here

[11:10:16] WARNING: C:/Jenkins/workspace/xgboost-win64_release_0.90/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
rmse for XGBRegressor:  129468.29847399698
